<a href="https://colab.research.google.com/github/faan03/ElGamal-threshold/blob/main/ElGamal_threshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensafeprime
import gensafeprime as gen
import random as random

class Zq:
    
    def __init__(self,q):
        self.q = q        

    def __modulo (self,a,b):
        return a%b # residuo div entera entre a y b

    def suma(self,a,b):
        r= self.__modulo((a+b),self.q)
        return r

    def resta(self,a,b):
        r = self.__modulo (a-b,self.q)
        return r

    def producto (self,a,b):
        r= self.__modulo(a*b,self.q)
        return r 

    def division (self,a,b):
        invM_b= self.invMulti(b)
        r= self.__modulo(a*invM_b,self.q)
        return r

    def potencia (self,base, potencia):   
        r= pow(base,potencia,self.q)
        return r

    def invMulti(self,a):
        r= self.__gcdExtended(a,self.q)[1]
        return r    

    def __gcdExtended(self,a, b): 
    # Base Case 
        if a == 0 :  
            return b,0,1             
        gcd,x1,y1 = self.__gcdExtended(b%a, a) 
        # Update x and y using results of recursive 
        # call 
        x = y1 - (b//a) * x1 
        y = x1      
        return gcd,x,y        

    def getRandomElement(self):
        return random.randint(0,self.q-1)

    def getRandomNotZeroElement(self):
        return random.randint(1,self.q-1)    

class shamirF:    
    
    def __init__(self,q, s,t):        
        self.s =s
        self.t =t
        self.fq = Zq(q)   
        
    def setAlfa(self,alfa):
        self.alfa=alfa
        
    def __hornerInZq(self,poly, n, x):     
        result = poly[0]       
        for i in range(1, n):             
            result = self.fq.suma(self.fq.producto(result,x) , poly[i]) 
        return result
    
    def __makePolynomial (self):
        # Seleccionar t-1 elementos aleatorios de zq
        self.poly = [self.fq.getRandomElement() for i in range(0,self.t-1) ]               
        self.poly.append(self.alfa)
        
    def generateParts(self):
        self.__makePolynomial()
        parts=[]
        for x in range (1, self.s+1):
            y = self.__hornerInZq(self.poly,len(self.poly),x) 
            parts.append((x,y))
        return parts    
        
    def makeSecret(self,tParts):
        alfa_g0=0
        for i in range (0,self.t) :       
            coef=1
            for j in range (0,self.t):
                if (j!=i):                                  
                    nu = self.fq.resta(0,tParts[j][0])
                    den = self.fq.resta(tParts[i][0],tParts[j][0])
                    coef = self.fq.producto(coef ,self.fq.division (nu,den))                                
            alfa_g0 = self.fq.suma (alfa_g0 , self.fq.producto(coef,tParts[i][1]))
        return alfa_g0            

class elgamal:
    
    def __init__(self,p,q):
        self.p=p
        self.q=q
        self.zp = Zq(p)
        self.zq = Zq(q)
        self.__getGenerator()              

    def __getGenerator(self):
        ##toma un elemento entre 1 y p-1        
        element= self.zp.getRandomElement()
        while ((self.zp.potencia (element,2)==1) or (self.zp.potencia(element,self.q)!=1)):
            element= self.zp.getRandomElement()
        self.generator = element

    def generateKeyPair(self):
        ##toma un elemento entre 1 - q-1
        sk =  random.randint(1,self.q-1)
        pk = self.zp.potencia (self.generator,sk)
        return (sk,pk)

    def E (self,pk,m): #m elemento en zp    
        k=  random.randint(1,self.q-1)    
        c1= self.zp.potencia(self.generator,k)
        c2 = self.zp.potencia(pk,k)
        c2 = self.zp.producto(c2,m)
        return (c1,c2)
    
    def D(self, sk,c): #c =c1,c2  
        c1= c[0]
        c2= c[1]
        w= self.zp.potencia(c1,sk)        
        return self.zp.division(c2, w)

    def DD(self,tParts,c2):
        w=1
        for i in range (0,len(tParts)) :       
            coef=1
            for j in range (0,len(tParts)):
                if (j!=i):                                  
                    nu = self.zq.resta(0,tParts[j][0])
                    den = self.zq.resta(tParts[i][0],tParts[j][0])
                    coef = self.zq.producto(coef ,self.zq.division (nu,den))                                
            w = self.zp.producto (w , self.zp.potencia(tParts[i][1],coef))            
            #alfa_g0            # w = v^alfa        
        return self.zp.division(c2, w) 



In [2]:
p = gen.generate(100)
q=((p-1)//2)

zp = Zq(p)

eg = elgamal(p,q)

(sk, pk) = eg.generateKeyPair()
s=10
t=5

#shamir se trabaja con q
sh = shamirF(q,s ,t)
sh.setAlfa(sk) # sk llave privada que seria alfa
parts = sh.generateParts() ## la llave privada se distribuye
print("la llave privada se parte en s partes y se  dsitribuye en s nodos ", parts)
m = zp.getRandomNotZeroElement()#mensaje
print("m= ",m)
print("Se cifra el mensaje con la llave publica del otro")
c = eg.E(pk,m)

print("D - >m ", (eg.D(sk,c)))
print("contactar los t nodos para descifra C")
#se contactan t partes

tIndex = random.sample(range(0,s),t)
print("tIndex ", tIndex)
rParts=[]
for ti in tIndex:
    print("nodo ", ti," se envia c (texto cifrado)")
    print("extrae c1 de C")
    c1 = c[0]
    (x,y) = parts[ti]
    rParts.append((x,zp.potencia(c1,y)))
    
print(" rParts : ", rParts)    
print(eg.DD(rParts,c[1]))



'''
m=5

c= eg.E(pk,m)
print(c)

print(eg.D(sk,c))
'''



la llave privada se parte en s partes y se  dsitribuye en s nodos  [(1, 512062098862260482048988034345), (2, 394870315209643586701516843409), (3, 514588602933835492736203618318), (4, 363936637460815351702829391661), (5, 175795775856508132591332081477), (6, 395389227389543918650874952994), (7, 96822565219535797847052003846), (8, 150002703020524081947477193639), (9, 53718918357532087436195214385), (10, 46922170435528948237407643546)]
m=  395269078980892489891022252712
Se cifra el mensaje con la llave publica del otro
D - >m  395269078980892489891022252712
contactar los t nodos para descifra C
tIndex  [7, 9, 4, 8, 1]
nodo  7  se envia c (texto cifrado)
extrae c1 de C
nodo  9  se envia c (texto cifrado)
extrae c1 de C
nodo  4  se envia c (texto cifrado)
extrae c1 de C
nodo  8  se envia c (texto cifrado)
extrae c1 de C
nodo  1  se envia c (texto cifrado)
extrae c1 de C
 rParts :  [(8, 200401401021553561259179433604), (10, 661665525197499869799497121443), (5, 849349588275257828721467903938),

'\nm=5\n\nc= eg.E(pk,m)\nprint(c)\n\nprint(eg.D(sk,c))\n'